In [1]:
from crystal_toolkit.helpers.layouts import Columns, Column
from dash import Dash
from jupyter_dash import JupyterDash
from pydefect.analyzer.calc_results import CalcResults
from pydefect.analyzer.dash_components.cpd_energy_dash import CpdEnergyComponent
from pydefect.chem_pot_diag.chem_pot_diag import ChemPotDiag, CpdPlotInfo, \
    CompositionEnergy
from pydefect.corrections.manual_correction import ManualCorrection
from pydefect.input_maker.defect_entry import DefectEntry
from pymatgen import Composition, Structure, Lattice
import dash_html_components as html
import crystal_toolkit.components as ctc
from dash.dependencies import Input, Output, State
import json

app = JupyterDash(__name__)
from vise.analyzer.band_edge_properties import BandEdge
#app = Dash()

comp_energies = {
    CompositionEnergy(Composition("Mg"), 0.0, "a"),
    CompositionEnergy(Composition("O"), 0.0, "a"),
    CompositionEnergy(Composition("MgO"), -1.0, "a")}
cpd = ChemPotDiag(comp_energies, target=Composition("MgO"))
cpd_plot_info = CpdPlotInfo(cpd)

structure = Structure(Lattice.cubic(1), species=["O"] * 2, coords=[[0]*3]*2)
defect_structure = Structure(Lattice.cubic(1), species=["O"] * 1, coords=[[0]*3])

common = dict(site_symmetry="1",
              magnetization=0.0,
              kpoint_coords=[[0]*3],
              kpoint_weights=[1.0],
              potentials=[0.0],
              vbm_info=BandEdge(0.0),
              cbm_info=BandEdge(1.0),
              fermi_level=0.0)

perfect = CalcResults(structure=structure,energy=0, **common)
defects = [CalcResults(structure=defect_structure, energy=1.0, **common),
           CalcResults(structure=defect_structure, energy=0.5, **common)]

de_common = dict(name="Va_O1",
                 structure=defect_structure, site_symmetry="1",
                 perturbed_structure=defect_structure, defect_center=[[0]*3])

defect_entries = [DefectEntry(charge=0, **de_common),
                  DefectEntry(charge=1, **de_common)]

corrections = [ManualCorrection(correction_energy=1.0),
               ManualCorrection(correction_energy=1.0)]

component = CpdEnergyComponent(cpd_plot_info=cpd_plot_info,
                               perfect=perfect,
                               defects=defects,
                               defect_entries=defect_entries,
                               corrections=corrections)

# example layout to demonstrate capabilities of component
pre = html.Pre(id='click-dataxxxxx',
               style={'border': 'thin lightgrey solid', 'overflowX': 'scroll'})
#my_layout = html.Div(Columns([Column(component.layout), Column(pre)]))
my_layout = html.Div([component.layout, pre])

@app.callback(
    Output('click-dataxxxxx', 'children'),
    [Input(component.id(), 'clickData')])
def display_click_data(clickData):
    return json.dumps(clickData, indent=2)

ctc.register_crystal_toolkit(app=app, layout=my_layout, cache=None)

app.run_server(port=8085)
#app.run_server(mode='inline')



Dash app running on http://127.0.0.1:8085/
